In [1]:
import numpy as np
import math
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt

In [57]:
#very preliminary attempt at making the IE DenseNet model into a class
class IntegralDenseNet:
    def __init__(self, learning_rate=0.01, leakyRelU_weight=0.1, optimizer=None):
        self.learning_rate = learning_rate
        self.leakyRelU_weight = leakyRelU_weight
        self.optimizer = tf.keras.optimizers.Adam(self.learning_rate) if optimizer is None else optimizer
    def prepTF(x):
        #converts numpy array to tensor and adds dimension to the front
        return tf.expand_dims((tf.convert_to_tensor(x, dtype=tf.float32), 0))                
    def fit(self, X, y, t0): #X has shape (batch, spatial shape, channels)
        self.t1 = t0+1
        self.filter_shape = (self.t1, *X.shape, 1) #time steps, spatial shape, in-channels, out-channels
        self.input_shape = (1, *X.shape) #batch size,(1), input spatial shape, in-channels

        rng = np.random.default_rng()
        iFilt = rng.standard_normal(self.filter_shape, dtype=np.float32)
        paramFilt = rng.standard_normal((self.t1,*self.filter_shape), dtype=np.float32) #layer from, layer to, spatial shape, in-channels, out-channels
        biasVec = rng.standard_normal((self.t1,1), dtype=np.float32)

        TFiFiltTensor = tf.compat.v1.get_variable("iFiltTensor", initializer=iFilt)
        self.iFilt = tf.Variable(TFiFiltTensor, name="iFilt")
        TFpFiltTensor = tf.compat.v1.get_variable("pFiltTensor", initializer=paramFilt)
        self.pFilt = tf.Variable(TFpFiltTensor, name="pFilt")
        self.b = tf.compat.v1.get_variable("biasVec", initializer=biasVec)
        
        model = (self.iFilt, self.pFilt, self.b)
        
        for i in np.arange(len(y)):
            tf_x = prepTF(X[i].astype(np.float32))
            tf_y = prepTF(y[i].astype(np.float32))
            optimize(self.optimizer, tf_x, *model, tf_y)   
    def trapezoidIntegral(vals, ds):
        pass1 = tf.reduce_sum(vals, axis=0)
        pass2 = tf.reduce_sum(vals[1:-1], axis=0)
        return ds/2.0*(pass1+pass2)
    def TFconv(image, filt, b):
        return tf.nn.convolution(input=image, filters=filt, padding="SAME") + b
    def leakyRelUInv(x):
        shape = x.shape
        zeros = tf.zeros(shape)
        mask = tf.math.greater_equal(x, zeros)
        weighted = x / leakyRelU_weight
        return tf.where(mask, x, weighted)
    def feedForward(self, x0, tape): #ideally this will only produce the output and will be tracked by the tape
        t1 = inFilt.shape[0]
        ds = 1.0/(t1-1)
        input_shape = x0.shape
        bzero = tf.zeros([inFilt.shape[4]])
        x = tf.TensorArray(tf.float32, size=0, dynamic_size=True, clear_after_read=False)
        u = tf.TensorArray(tf.float32, size=0, dynamic_size=True, clear_after_read=False)

        tape.watch(x0)
        u0 = leakyRelUInv(x0)
        u = u.write(0,TFconv(u0, inFilt[0], bzero))

        for h in range(t1-1):
            x = x.write(h, tf.nn.leaky_relu(u.read(h), alpha=leakyRelU_weight))
            zInt = tf.TensorArray(tf.float32, size=0, dynamic_size=True)
            for s in range(h+1):
                zInt = zInt.write(s, TFconv(x.read(s), pFilt[s,h], b[h] / ((h+1)*ds)))
            z = trapezoidIntegral(zInt.stack(), ds)
            u = u.write(h+1, z + TFconv(u0, inFilt[h+1], bzero))
        x = x.write(t1-1,tf.nn.leaky_relu(u.read(t1-1), alpha=leakyRelU_weight))

        loss = tf.math.square(x.read(t1-1) - y)
        gradients = tape.gradient(loss, [pFilt, inFilt, b])

        return x.read(t1-1), tf.math.reduce_sum(loss), gradients
    def optimize(optimizer, xIn, y): #ideally this can be called only when descending gradient, and feedForward can be called for output
        with tf.GradientTape() as tape:
            output = TFuSoln(xIn, tape)
            loss = tf.math.square(output - y)
            gradients = tape.gradient(loss, [pFilt, inFilt, b])
            
        optimizer.apply_gradients(zip(grad, [pFilt, iFilt, bias]))
        return loss
            
            

In [4]:
TFuSoln = tf.function(uSoln)

In [43]:
np.set_printoptions(suppress=True)

nn = IntegralDenseNet()
X = np.random.random((10,5,5,1))
y = np.random.random((10,5,5))

In [5]:
learning_rate = 0.01
leakyRelU_weight = 0.1
opt = tf.keras.optimizers.Adam(learning_rate)
input_shape = (1,5,5)
filter_shape = (1,1,3,3)
t0 = 5

model = make_model(t0, input_shape, filter_shape)
y = prepped_output(np.random.rand(*input_shape).astype(np.float32))


iterations = 1
losses = []
for i in np.arange(iterations):
    npx, npy = make_xy_pair(input_shape)
    x = prepped_input(npx.astype(np.float32))
    y = prepped_output(npy.astype(np.float32))
    losses.append(optimize(opt, x, *model, y))